In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.input import loadMidiFile
from src.io.output import pianoRoll, playPrettyMidi
from src import analysis
from src.adaptation import Adaptation
from src.datatypes.melody_data import MelodyData
from src.evaluation import Evaluation
from src.db.reference_sets import get_normalization_values_of_ref_set
from definitions import SequenceType
from src.io.conversion import pretty_midi_to_music21, music21_to_pretty_midi

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Instantiate Adaptation Object


In [2]:
a = Adaptation()

## Print available adaptation operations

In [3]:
for op in a.available_operations:
    print(op.__name__, ":", op.__doc__)

SameNoteOffsetsOperation : Changes all note offsets at a certain beat position to a note offset that also occures in the control sequence at that beat position.
StartAndEndOnCTOperation : Creates a chord based on the estimated key of the control sequence and transposes the first and last note of the base sequence so they are chord tones (e.g. 'c', 'e' or 'g' for a C major chord).
TransposeNotesOperation : Estimates the key of both sequences and transposes every single note of the base sequence separately to the closest pitch that fits the key of the control sequence.
TransposeSequenceOperation : Estimates the key of both sequences and transposes the base sequence to be in the same key as the control sequence.


## Construct a pipeline

In [4]:
a.construct_pipeline(['SameNoteOffsetsOperation'])
a.pipeline.get_operations()

['SameNoteOffsetsOperation']

## Load 2 Melodies to emulate input and gen_base
+ print the notes of melody 2 (the base for adaptation)

In [5]:
midi1 = loadMidiFile('../midi/examples/monophonic/twinkle1_4b.mid')
midi2 = loadMidiFile('../midi/examples/monophonic/mii1_4b.mid')

mel1 = MelodyData(midi1, SequenceType.REC_INPUT, {})
mel2 = MelodyData(midi2, SequenceType.GEN_BASE, {})

print("length:", len(mel2.sequence.instruments[0].notes))
mel2.sequence.instruments[0].notes

length: 18
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pretty_midi\pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


[Note(start=0.005208, end=0.214583, pitch=66, velocity=73),
 Note(start=0.514583, end=0.610417, pitch=69, velocity=69),
 Note(start=0.761458, end=0.859375, pitch=73, velocity=68),
 Note(start=1.259375, end=1.355208, pitch=69, velocity=68),
 Note(start=1.752083, end=1.850000, pitch=66, velocity=68),
 Note(start=2.002083, end=2.116667, pitch=62, velocity=67),
 Note(start=2.266667, end=2.364583, pitch=62, velocity=62),
 Note(start=2.514583, end=2.610417, pitch=62, velocity=63),
 Note(start=3.752083, end=4.029167, pitch=61, velocity=62),
 Note(start=4.002083, end=4.116667, pitch=62, velocity=57),
 Note(start=4.266667, end=4.364583, pitch=66, velocity=68),
 Note(start=4.514583, end=4.610417, pitch=69, velocity=69),
 Note(start=4.761458, end=4.859375, pitch=73, velocity=68),
 Note(start=5.259375, end=5.355208, pitch=69, velocity=68),
 Note(start=5.752083, end=5.850000, pitch=66, velocity=68),
 Note(start=6.002083, end=6.760417, pitch=76, velocity=73),
 Note(start=6.761458, end=7.009375, pitc

## Adapt the melody and print the results

In [6]:
result, control = a.adapt(mel2, mel1)
print("length:", len(result.sequence.instruments[0].notes))
result.sequence.instruments[0].notes

adapted 1/96
inserted Note(start=0.000000, end=0.209375, pitch=66, velocity=73)
length: 18


[Note(start=0.000000, end=0.209375, pitch=66, velocity=73),
 Note(start=0.500000, end=0.595833, pitch=69, velocity=69),
 Note(start=0.500000, end=0.597917, pitch=73, velocity=68),
 Note(start=1.000000, end=1.095833, pitch=69, velocity=68),
 Note(start=1.500000, end=1.597917, pitch=66, velocity=68),
 Note(start=0.000000, end=0.114583, pitch=62, velocity=67),
 Note(start=0.000000, end=0.097917, pitch=62, velocity=62),
 Note(start=0.500000, end=0.595833, pitch=62, velocity=63),
 Note(start=1.500000, end=1.777083, pitch=61, velocity=62),
 Note(start=0.000000, end=0.114583, pitch=62, velocity=57),
 Note(start=0.000000, end=0.097917, pitch=66, velocity=68),
 Note(start=0.500000, end=0.595833, pitch=69, velocity=69),
 Note(start=0.500000, end=0.597917, pitch=73, velocity=68),
 Note(start=1.000000, end=1.095833, pitch=69, velocity=68),
 Note(start=1.500000, end=1.597917, pitch=66, velocity=68),
 Note(start=0.000000, end=0.758333, pitch=76, velocity=73),
 Note(start=0.500000, end=0.747917, pitc

In [7]:
result.analysis

{}

In [8]:
control.analysis

{'note_offsets_per_beat': [{0.0}, {0.0}, {0.0}, {0.0}]}

In [9]:
from src.utils.melodies import find_closest

stream = pretty_midi_to_music21(result.sequence)
i = 0
for n in stream.parts[0].notes:
    beat = int(n.offset) % 4
    relative_offset = float(n.offset % 1)
    closest_allowed_offset = find_closest((0.0, 0.5), relative_offset)
    n.offset = int(n.offset) + closest_allowed_offset
    
pm = music21_to_pretty_midi(stream)
pm.instruments[0].notes

[Note(start=0.000000, end=0.209375, pitch=66, velocity=73),
 Note(start=0.500000, end=0.595833, pitch=69, velocity=69),
 Note(start=0.500000, end=0.597917, pitch=73, velocity=68),
 Note(start=1.000000, end=1.095833, pitch=69, velocity=68),
 Note(start=1.500000, end=1.597917, pitch=66, velocity=68),
 Note(start=0.000000, end=0.114583, pitch=62, velocity=67),
 Note(start=0.000000, end=0.097917, pitch=62, velocity=62),
 Note(start=0.500000, end=0.595833, pitch=62, velocity=63),
 Note(start=1.500000, end=1.777083, pitch=61, velocity=62),
 Note(start=0.000000, end=0.114583, pitch=62, velocity=57),
 Note(start=0.000000, end=0.097917, pitch=66, velocity=68),
 Note(start=0.500000, end=0.595833, pitch=69, velocity=69),
 Note(start=0.500000, end=0.597917, pitch=73, velocity=68),
 Note(start=1.000000, end=1.095833, pitch=69, velocity=68),
 Note(start=1.500000, end=1.597917, pitch=66, velocity=68),
 Note(start=0.000000, end=0.758333, pitch=76, velocity=73),
 Note(start=0.500000, end=0.747917, pitc

# Evaluation

In [10]:
normalization_values = get_normalization_values_of_ref_set(1)
normalization_values

{'avg_ioi': 0.040211928,
 'avg_pitch_interval': 0.4,
 'note_count': 2,
 'note_length_histogram': 0.2018644384,
 'note_length_transition_matrix': 4.3588989435,
 'pitch_class_histogram': 0.2924397324,
 'pitch_class_histogram_per_bar': 0.2924397324,
 'pitch_class_transition_matrix': 4.2426406871,
 'pitch_count': 1,
 'pitch_range': 2,
 'avg_pitch_interval_distance': nan}

### Result Evaluation

In [11]:
eval = Evaluation(normalization_values)
evaluation = eval.evaluate_similarity(result.sequence, control.sequence)
evaluation

{'absolute': {'pitch_count': 2.0,
  'pitch_class_histogram': 0.4772594526094087,
  'pitch_class_transition_matrix': 3.605551275463989,
  'avg_pitch_interval': 2.7330316742081444,
  'pitch_range': 6.0,
  'note_count': 4.0,
  'note_length_histogram': 1.1065666703449761,
  'note_length_transition_matrix': 13.076696830622021,
  'avg_ioi': 0.45022624434389136},
 'normalized': {'pitch_count': 2.0,
  'pitch_class_histogram': 1.6319925089953637,
  'pitch_class_transition_matrix': 0.8498365856026605,
  'avg_pitch_interval': 6.832579185520361,
  'pitch_range': 3.0,
  'note_count': 2.0,
  'note_length_histogram': 5.481731597282546,
  'note_length_transition_matrix': 3.0000000000279936,
  'avg_ioi': 11.196335682882237}}